In [1]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'prepare/mesolitica-tpu.json'

In [2]:
from google.cloud import storage
client = storage.Client()
bucket = client.bucket('mesolitica-tpu-general')

In [3]:
best = '321300'
directory = 't5-small-v2'
!rm -rf output out {directory}
!mkdir {directory}

In [4]:
model = best

blob = bucket.blob(f'{directory}/model.ckpt-{model}.data-00000-of-00002')
blob.download_to_filename(f'{directory}/model.ckpt-{model}.data-00000-of-00002')

blob = bucket.blob(f'{directory}/model.ckpt-{model}.data-00001-of-00002')
blob.download_to_filename(f'{directory}/model.ckpt-{model}.data-00001-of-00002')

blob = bucket.blob(f'{directory}/model.ckpt-{model}.index')
blob.download_to_filename(f'{directory}/model.ckpt-{model}.index')

blob = bucket.blob(f'{directory}/model.ckpt-{model}.meta')
blob.download_to_filename(f'{directory}/model.ckpt-{model}.meta')

blob = bucket.blob(f'{directory}/checkpoint')
blob.download_to_filename(f'{directory}/checkpoint')

blob = bucket.blob(f'{directory}/operative_config.gin')
blob.download_to_filename(f'{directory}/operative_config.gin')

In [5]:
with open(f'{directory}/checkpoint', 'w') as fopen:
    fopen.write(f'model_checkpoint_path: "model.ckpt-{model}"')

In [6]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds
import t5

In [8]:
# !pip3 install t5==0.5.0

In [9]:
model = t5.models.MtfModel(
    model_dir=directory,
    tpu=None,
    tpu_topology=None,
    model_parallelism=1,
    batch_size=1,
    sequence_length={"inputs": 512, "targets": 512},
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=3,
    iterations_per_loop=100,
    mesh_shape="model:1,batch:1", 
    mesh_devices=["cpu:0"]
)

In [10]:
string = """
Amanah Kedah berpendapat jika ada Adun Pakatan Harapan atau Bersatu negeri itu mahu berpaling tadah memberikan sokongan kepada kumpulan Muafakat Nasional, mereka perku membuat kenyataan rasmi mengenainya.

Pengerusi Amanah Kedah, Phahrolrazi Mohd Zawawi, berkata disebabkan tiada mana-mana Adun membuat kenyataan berhubung isu itu maka kerajaan negeri berpendapat tiada apa-apa yang berlaku.

Ditemui media selepas mengadakan pertemuan tertutup lebih sejam dengan Menteri Besar, Mukhriz Mahathir, hari ini Phahrolrazi berkata pihaknya juga mendapati kerajaan negeri masih berfungsi seperti biasa.

"Kami bincang keadaan semasa, ada juga kita sentuh (cubaan menukar kerajaan negeri), tetapi kita lihat kerajaan masih berfungsi.

"Tidak ada apa-apa kenyataan dari pihak sana (pembangkang) bahawa mereka sudah cukup majoriti setakat ini," katanya seperti dipetik BH Online.

Spekulasi mengenai pertukaran kerajaan menjadi kencang sejak semalam ekoran berlaku pertemuan tertutup pemimpin PAS dan Umno Kedah di Alor Setar semalam.

Turut hadir Setiausaha Agung PAS yang juga Menteri di Jabatan Perdana Menteri, Takiyuddin Hassan, dan Menteri Besar Terengganu, Dr Ahmad Samsuri Mokhtar.  

Cuba jatuhkan sejak dulu

Perkembangan itu berlaku kesan tindakan PKR memecat dan menggantung sejumlah anggota mereka baru-baru ini dan dipercayai memberi kesan terhadap pendirian wakil rakyat parti itu di Kedah.

Turut disebut-sebut akan beralih arah dalam perjalanan politik mereka ialah Adun Bersatu.

Untuk rekod berdasarkan pecahan parti PAS menguasai kerusi terbesar dalam DUN dan lazimnya pemimpin parti itu akan menjadi pilihan menjadi menteri besar jika berlaku pertukaran kerajaan.

Menurut Phahrolrazi, jika ada mana-mana wakil rakyat Bersatu atau PH mahu melompat, mereka wajar menyatakannya secara rasmi.

Tanpa kenyataan begitu, katanya, Amanah beranggapan isu perubahan kerajaan negeri masih bersifat spekulasi.

Timbalan Pengerusi Amanah Kedah, Dr Ismail Salleh, pula berkata ada kemungkinan Adun Bersatu, PH atau exco negeri tu yang sudah diumpan untuk membelakangkan mandat rakyat.

Beliau yang juga exco Kedah berkata memang sejak dulu lagi PAS cuba menjatuhkan kerajaan negeri dengan memujuk Adun PH serta Bersatu bertindak seperti rakan mereka di Perak, Johor dan Selangor.
"""

In [11]:
# https://harakahdaily.net/index.php/2020/05/22/apa-ph-buat-isu-kenaikan-harga-barang/

string2 = """
SEKARANG PH tengah mainkan isu harga barang untuk memburukkan Kerajaan PN. Kata Ahli Majlis Setiausaha PH (Saifuddin Nasution, Khalid Samad dan Ong Kian Ming) harga barang mentah dan basah menunjukkan ada peningkatan di banyak tempat. Ini membebankan rakyat kata mereka.

Saya setuju, Kerajaan PN perlu pantau dan ambil apa-apa tindakan yang perlu jika ada peningkatan harga barang basah atau mentah.

Tapi dalam masa yang sama saya fikir elok juga kalau kita imbas balik apa pula tindakan yang diambil oleh Kerajaan PH ketika rakyat tertekan dengan harga barang yang melonjak gila-gila masa mereka jadi Kerajaan Persekutuan selama 22 bulan dulu.

Mari kita semak balik.

(1) Kenaikan harga telur
Saifuddin Nasution – “Kementerian akan siasat.”
Salahuddin Ayub – “Ianya adalah perkara biasa disebabkan kenaikan bahan makanan ayam.”

(2) Kenaikan harga bawang 3 kali ganda
Saifuddin Nasution – “KPNHEP akan pantau kenaikan ini. Rakyat perlu tukar kepada bawang Holland.”

(3) Kenaikan harga ikan kembung
Salahuddin Ayub – “Saya pelik mengapa harga ikan kembung naik sedangkan ada 7 bilion ikan kembung dalam laut.”

(4) Kenaikan harga barangan kawalan
Saifuddin Nasution – “KPNHEP tidak menerima sebarang aduan kenaikan harga barang kawalan. Kita akan siasat.”

(5) Kenaikan harga barang mentah dan basah menjelang Aidilfitri 2019
Saifuddin Nasution – “KPNHEP puas hati dengan trend semasa harga barang keperluan. Tiada kenaikan mendadak (kenaikan sedikit demi sedikit itu perkara biasa).”

(6) Kenaikan harga barang keperluan pada penghujung 2018 dan sepanjang 2019
Lim Guan Eng – “Kenaikan harga barang disebakan banyak faktor seperti penyusutan nilai Ringgit, musim perayaan, kegiatan penyeludupan, kemarau di negara pengeluar selain sikap peniaga yang mahu mengaut keuntungan berlebihan. Ianya tidak dapat dielakkan.”

Yang saya listkan di atas hanya sebahagian kecil daripada kenyataan Menteri-menteri PH. Ada banyak lagi.

Kesimpulannya, PH tak buat apa-apa pun berkaitan kenaikan harga barang keperluan yang meresahkan rakyat semasa mereka jadi Kerajaan. Tapi sekarang tiba-tiba pula mereka jadi prihatin dan mahu jadi hero rakyat.
"""

In [12]:
# https://www.malaysiakini.com/news/526581

string3 = """
KATA MP | Saya membaca kenyataan beberapa orang penganalisis politik seperti yang dilaporkan dalam media - misalnya di Malaysiakini - mengenai perkembangan semasa politik dalam negara.

Mereka, antara lain, menyarankan agar Pakatan Harapan (PH) memberi tumpuan kepada PRU15 untuk mengambil kerajaan dan bukannya melakukan hal itu pada masa sekarang.

Seorang pemimpin Umno pula dilaporkan oleh akhbar Sinar Harian dengan bangga menggunakan ucapan titah yang di-Pertuan Agong semasa hari perasmian sidang parlimen pada 18 Mei lalu bagi menunjukkan kononnya kerajaan yang ada sekarang ialah "kerajaan Tuanku".

Pertamanya, saya ingin ucapkan terima kasihlah kepada para penganalisis politik tersebut atas analisis mereka. Maaf - dan dengan penuh hormat - jika saya katakan bahawa analisis anda semua bernilai amat amat rendah dan hampir mencapai tahap tidak ada nilai.

Saya tidak pasti di mana anda semua simpan disiplin ilmu anda.

Keduanya, sekiranya kritikan anda semua itu benar-benar jujur, cadangan anda semua (bahawa perlu tunggu hingga PRU15 untuk ambil alih kerajaan) itu sewajarnya ditujukan kepada Perikatan Nasional (PN) yang membentuk kerajaan sekarang tanpa sebarang mandat rakyat dalam sebarang pilihan raya.

Ia diakui sendiri oleh Perdana Menteri Muhyiddin Yassin semasa berucap secara langsung di televisyen baru-baru ini. Bagi saya ia sesuai dinobatkan sebagai Ucapan Mak Cik Kiah.

Dibentuk secara sah, bermoral

Ketiga, jika PH kembali berkuasa pada masa ini, hakikatnya ia berkuasa semula dengan mandat rakyat yang diberi oleh rakyat menerusi PRU14.

Mandat tersebut tidak pernah ditarik balik! Bukankah mandat itu untuk tempoh lima tahun? Justeru, siapa yang tidak hormat kepada mandat rakyat tersebut?

Keempat, meminta agar PH menunggu sehingga PRU15 nampaknya berdasarkan andaian atau premis bahawa kerajaan PN ini wajar dibiarkan mentadbir beberapa tahun lagi.

Pun begitu ada beberapa fakta yang perlu diperhatikan, iaitu:

Ia merupakan kerajaan yang memiliki saiz kabinat yang lebih besar berbanding PH.
Kerajaan PN meriah mengagih-agihkan ghanimah kepada aktor-aktor politik seperti pelantikan sebagai duta khas bertaraf menteri dan pelantikan dalam syarikat berkaitan kerajaan (GLC) bagi 'membeli' kesetiaan
Ia merupakan kerajaan yang lebih teruja 'membebaskan' insan yang terpalit dengan skandal mega 1MDB.
Sedarkah para penganalisis berkenaan mengenai semua realiti ini, selain banyak isu lain membabitkan imej negara, ekonomi, pelaburan asing, keyakinan pelabur dan sebagainya?

Kelima, kerajaan PH yang ditumbangkan oleh PN sekarang adalah kerajaan yang tidak ada sedikit pun 'daki' ketakabsahan (legitimacy) sama ada dari sudut perlembagaan, proses demokrasi atau dari sudut moral dan agama.

Di mana suara mereka sebelum ini

Tiada sedikit pun elemen pengkhianatan dalam pembentukan kerajaan PH. Ia dibentuk secara sah, secara bermoral dan beretika sambil menjunjung tinggi semangat dan ruh Perlembagaan Persekutuan.

Ia adalah sepenuhnya kerajaan rakyat yang mewakili semua bangsa, etnik dan agama.

Ya, benar, kerajaan PH itu tidaklah sempurna. Ada kelemahan di sini sana. Namun siapa yang boleh menafikan kesungguhan kerajaan PH bagi membetulkan kerosakan amat teruk yang ditinggalkan oleh kerajaan sebelum ini?

Kesungguhan itu boleh dilihat oleh ramai orang di dalam atau di luar negara. Semasa saya bertugas di luar negara atas kapasiti anggota pentadbiran kerajaan PH, ramai rakan sejawat saya di luar yang bukan sahaja memuji kita malah lebih daripada itu mereka berjanji untuk membantu.

Saya masih ingat mantan perdana menteri - Dr Mahathir Mohamad - sering mengatakan bahawa hanya setelah beberapa bulan PH mengambil alih tampuk pemerintahan negara, usaha menukar imej negara ini yang dikenali sebagai negara kleptokrat mula menunjukkan hasilnya.

Keenam, saya tertanya-tanya di mana suara para penganalisis politik itu semua ketika PN mengambil alih kerajaan PH secara tidak bermaruah hingga digelar kerajaan pintu belakang atau kerajaan tebuk atap?

Ya, di mana suara mereka ketika Langkah Sheraton digerakkan?

Kenapa - pada masa itu - kita tidak dengar mereka menasihati PN agar menunggu sehingga PRU15 untuk menjadi kerajaan?

Tentang isu "kerajaan Tuanku" atau "kerajaan beta" itu, maaflah, takkanlah serendah itu tahap politik mereka? Kenapa terlalu paranoid dengan lafaz itu?

Dalam sistem demokrasi ala Westminster yang menjunjung prinsip raja berperlembagaan, tiada sebarang keanehan pun dengan istilah tersebut. Ia fenomena yang lumrah. Saya rasa aneh apabila ada yang sanggup mengeksploitasi istilah berkenaan.

Amat tidak wajar dan amat tidak beretika sama sekali untuk mengheret institusi diraja - sama seperti tidak wajar mengeksploitasi isu agama dan perkauman - demi kelangsungan jangka hayat politik yang kita tahu sedang begitu nazak sekarang.
"""

In [13]:
string4 = """
ORANG ramai tidak perlu panik atau bimbang dengan berita tular di media sosial mendakwa ayam yang dibeli di pasar raya menjadi `ejen' pembawa virus Covid-19 dan boleh membahayakan nyawa.

Ini berdasarkan kenyataan dikeluarkan Jabatan Perkhidmatan Veteriner Putrajaya (JPVP) pada 9 Mei lalu, menjelaskan bahawa ayam ternakan tidak boleh dijangkiti virus Covid-19, walaupun pekerja kilang berkenaan atau mereka yang mengurus atau memproses ayam mempunyai kontak Covid-19.

Jurucakap jabatan itu berkata, virus covid-19 juga tidak mampu bertahan lama pada daging ayam sekiranya pekerja syarikat ayam terbabit disahkan positif Covid-19.

Katanya, dalam proses penyembelihan dan pemprosesan yang dilakukan terhadap ayam, virus Covid-19 yang mungkin ada pada ayam akan mati dan ayam berkenaan selamat dimakan.

"Kenyataan ini juga disokong dengan laporan dikeluarkan Pertubuhan Kesihatan Sedunia (WHO) pada 7 April 2020, yang menjelaskan bahawa makanan yang diproses selamat dimakan.

"Malah, laporan itu juga mengesahkan bahawa ujian Covid-19 yang dibuat terhadap ayam dari negara China menunjukkan semuanya negatif penyakit berkenaan," katanya di sini, tadi.

Isu kebimbangan orang ramai mengenai kedudukan ayam timbul selepas wujud kluster baharu Covid-19 di sebuah kilang pemprosesan ayam di Pedas Rembau Negeri Sembilan, pada 2 Mei lalu.

Kebimbangan itu semakin bertambah apabila isu berkenaan viral dalam laman sosial hingga mendapat pelbagai pandangan daripada netizen.

Dalam laporan yang dikeluarkan Kementerian Kesihatan Malaysia (KKM) peringkat awal mendapati 60 pekerja loji pemprosesan ayam di kilang terbabit positif Covid-19.

Difahamkan kilang berkenaan adalah loji pemprosesan ayam komersial di bawah seliaan JPV dan dikatakan memproses dan membekalkan sehingga 54,000 ekor ayam sehari daripada jumlah keseluruhan bekalan ayam segar seluruh negara purata sebanyak 1.6 juta ekor sehari.

Sumber ayam hidup diperoleh daripada 79 ladang ayam kontrak di bawah kawal selia syarikat dan 23 ladang persendirian.

Syarikat itu juga membekalkan ayam yang telah diproses ke beberapa pasar raya.

Kluster baharu Covid-19 di kilang berkenaan mula dikesan apabila seorang kakitangan syarikat mengalami gejala demam, batuk, selsema, sesak nafas dan sakit kepala pada 5 April lalu.

Pekerja terbabit telah menjalani saringan ujian Covid-19 pada 10 April dan keputusan ujian disahkan positif dua hari kemudian.

Sehingga 8 Mei lalu, seramai 786 orang kakitangan dan ahli keluarga kilang berkenaan telah dijalankan saringan ujian Covid-19 dan daripada jumlah terbabit, 60 didapati positif Covid-19.

Seramai 286 negatif dan 440 orang masih menunggu keputusan dan pekerja yang disahkan popsitif Covid-19 telah dikuarantin.

Susulan itu, KKM mengeluarkan arahan menghentikan operasi kilang terbabit selama 14 hari berkuatkuasa dari 7 hingga 20 Mei ini.

Dalam laporan yang dikeluarkan KKM semalam, terdapat pertambahan 43 kes, membabitkan warga pekerja kilang berkenan menjadikan jumlah kes positif Covid-19 kepada 131 kes.

"Daripada jumlah itu 122 kes melibatkan bukan warganegara," kata Ketua Pengarah Kesihatan, Dr Noor Hisham Abdullah.
"""

In [14]:
import re

# minimum cleaning, just simply to remove newlines.
def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

string = cleaning(string)
string2 = cleaning(string2)
string3 = cleaning(string3)
string4 = cleaning(string4)

# with tf.io.gfile.GFile('test.txt', "w") as f:
#     f.write("ringkasan: %s\n" % string2)
#     f.write("ringkasan: %s\n" % string3)
#     f.write("ringkasan: %s\n" % string4)

In [15]:
!rm -rf output/*

In [16]:
import gin

from t5.data import sentencepiece_vocabulary

DEFAULT_SPM_PATH = 'prepare/sp10m.cased.ms-en.model'
DEFAULT_EXTRA_IDS = 100
model_dir = directory

def get_default_vocabulary():
    return sentencepiece_vocabulary.SentencePieceVocabulary(
      DEFAULT_SPM_PATH, DEFAULT_EXTRA_IDS)

with gin.unlock_config():
    gin.parse_config_file(t5.models.mtf_model._operative_config_path(model_dir))
    gin.bind_parameter("Bitransformer.decode.beam_size", 1)
    gin.bind_parameter("Bitransformer.decode.temperature", 0)
    gin.bind_parameter("utils.get_variable_dtype.slice_dtype", "float32")
    gin.bind_parameter(
        "utils.get_variable_dtype.activation_dtype", "float32")
    
vocabulary = t5.data.SentencePieceVocabulary(DEFAULT_SPM_PATH)
estimator = model.estimator(vocabulary, disable_tpu=True)

INFO:tensorflow:Using config: {'_model_dir': 't5-small-v2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': None, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=None, num_cores_per_replica=1, per_host_input_for_training=4, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None, eval_training_inp

In [17]:
estimator.__dict__

{'_log_every_n_steps': 1,
 '_config': <tensorflow_estimator.python.estimator.tpu.tpu_config.RunConfig at 0x7f8ddf8b9b00>,
 '_train_distribution': None,
 '_eval_distribution': None,
 '_model_dir': 't5-small-v2',
 '_session_config': allow_soft_placement: true
 isolate_session_state: true,
 '_device_fn': None,
 '_model_fn': <function tensorflow_estimator.python.estimator.tpu.tpu_estimator.TPUEstimator._augment_model_fn.<locals>._model_fn(features, labels, mode, config, params)>,
 '_params': {},
 '_warm_start_settings': None,
 '_iterations_per_training_loop': IterationsPerLoopCounter(value=100, unit='count'),
 '_log_every_n_secs': None,
 '_ctx': <tensorflow_estimator.python.estimator.tpu.tpu_context._InternalTPUContext at 0x7f8ddf8b9b38>,
 '_export_to_cpu': True,
 '_export_to_tpu': False,
 '_export_saved_model_api_version': <ExportSavedModelApiVersion.V1: 1>,
 '_is_input_fn_invoked': None,
 '_rendezvous': {}}

In [18]:
import os

checkpoint_step = t5.models.mtf_model._get_latest_checkpoint_from_dir(model_dir)
model_ckpt = "model.ckpt-" + str(checkpoint_step)
checkpoint_path = os.path.join(model_dir, model_ckpt)
checkpoint_step, model_ckpt, checkpoint_path

(321300, 'model.ckpt-321300', 't5-small-v2/model.ckpt-321300')

In [19]:
from mesh_tensorflow.transformer import dataset as transformer_dataset

def serving_input_fn():
    inputs = tf.placeholder(
            dtype=tf.string,
            shape=[None],
            name="inputs")

    batch_size = tf.shape(inputs)[0]
    padded_inputs = tf.pad(inputs, [(0, tf.mod(-tf.size(inputs), batch_size))])
    dataset = tf.data.Dataset.from_tensor_slices(padded_inputs)
    dataset = dataset.map(lambda x: {"inputs": x})
    dataset = transformer_dataset.encode_all_features(dataset, vocabulary)
    dataset = transformer_dataset.pack_or_pad(
        dataset=dataset,
        length=model._sequence_length,
        pack=False,
        feature_keys=["inputs"]
    )
    dataset = dataset.batch(tf.cast(batch_size, tf.int64))
    features = tf.data.experimental.get_single_element(dataset)
    return tf.estimator.export.ServingInputReceiver(
        features=features, receiver_tensors=inputs)

out = estimator.export_saved_model('output', serving_input_fn, checkpoint_path=checkpoint_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:feature inputs : Tensor("Reshape:0", shape=(1, 512), dtype=int32)
Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:

INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/k                  size 262144       slice_size 262144       Shape[d_model=512, heads=512]                               
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/o                  size 262144       slice_size 262144       Shape[heads=512, d_model=512] 

INFO:tensorflow:Variable decoder/block_002/layer_001/EncDecAttention/q                size 262144       slice_size 262144       Shape[d_model=512, heads=512]                               
INFO:tensorflow:Variable decoder/block_002/layer_001/EncDecAttention/v                size 262144       slice_size 262144       Shape[d_model=512, heads=512]                               
INFO:tensorflow:Variable decoder/block_002/layer_001/layer_norm/scale                 size 512          slice_size 512          Shape[d_model=512]                                          
INFO:tensorflow:Variable decoder/block_002/layer_002/DenseReluDense/wi/kernel         size 1048576      slice_size 1048576      Shape[d_model=512, d_ff=2048]                               
INFO:tensorflow:Variable decoder/block_002/layer_002/DenseReluDense/wo/kernel         size 1048576      slice_size 1048576      Shape[d_ff=2048, d_model=512]                               
INFO:tensorflow:Variable decoder/block_002/layer_002/la

INFO:tensorflow:Variable decoder/block_005/layer_002/layer_norm/scale                 size 512          slice_size 512          Shape[d_model=512]                                          
INFO:tensorflow:Variable decoder/final_layer_norm/scale                               size 512          slice_size 512          Shape[d_model=512]                                          
INFO:tensorflow:Variable encoder/block_000/layer_000/SelfAttention/k                  size 262144       slice_size 262144       Shape[d_model=512, heads=512]                               
INFO:tensorflow:Variable encoder/block_000/layer_000/SelfAttention/o                  size 262144       slice_size 262144       Shape[heads=512, d_model=512]                               
INFO:tensorflow:Variable encoder/block_000/layer_000/SelfAttention/q                  size 262144       slice_size 262144       Shape[d_model=512, heads=512]                               
INFO:tensorflow:Variable encoder/block_000/layer_000/Se

INFO:tensorflow:Variable encoder/block_005/layer_000/SelfAttention/o                  size 262144       slice_size 262144       Shape[heads=512, d_model=512]                               
INFO:tensorflow:Variable encoder/block_005/layer_000/SelfAttention/q                  size 262144       slice_size 262144       Shape[d_model=512, heads=512]                               
INFO:tensorflow:Variable encoder/block_005/layer_000/SelfAttention/v                  size 262144       slice_size 262144       Shape[d_model=512, heads=512]                               
INFO:tensorflow:Variable encoder/block_005/layer_000/layer_norm/scale                 size 512          slice_size 512          Shape[d_model=512]                                          
INFO:tensorflow:Variable encoder/block_005/layer_001/DenseReluDense/wi/kernel         size 1048576      slice_size 1048576      Shape[d_model=512, d_ff=2048]                               
INFO:tensorflow:Variable encoder/block_005/layer_001/De

In [20]:
config = tf.ConfigProto()
config.allow_soft_placement = True
sess = tf.Session(config = config)
meta_graph_def = tf.saved_model.loader.load(
        sess,
        [tf.saved_model.tag_constants.SERVING],
        out)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from output/1626957947/variables/variables


In [21]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'out/model.ckpt')

'out/model.ckpt'

In [22]:
strings = [
    n.name
    for n in tf.get_default_graph().as_graph_def().node
    if ('encoder' in n.op
    or 'decoder' in n.name
    or 'shared' in n.name
    or 'inputs' in n.name
    or 'output' in n.name
    or 'SentenceTokenizer' in n.name
    or 'self/Softmax' in n.name)
    and 'adam' not in n.name
    and 'Assign' not in n.name
]

In [23]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names,
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [24]:
freeze_graph('out', strings)

INFO:tensorflow:Restoring parameters from out/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 212 variables.
INFO:tensorflow:Converted 212 variables to const ops.
7064 ops in the final graph.


In [25]:
import struct

unknown = b'\xff\xff\xff\xff'

def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    for node in graph_def.node:
        
        if node.op == 'RefSwitch':
          node.op = 'Switch'
          for index in xrange(len(node.input)):
            if 'moving_' in node.input[index]:
              node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
          node.op = 'Sub'
          if 'use_locking' in node.attr: del node.attr['use_locking']
        elif node.op == 'AssignAdd':
          node.op = 'Add'
          if 'use_locking' in node.attr: del node.attr['use_locking']
        elif node.op == 'Assign':
          node.op = 'Identity'
          if 'use_locking' in node.attr: del node.attr['use_locking']
          if 'validate_shape' in node.attr: del node.attr['validate_shape']
          if len(node.input) == 2:
            node.input[0] = node.input[1]
            del node.input[1]
            
        if 'Reshape/shape' in node.name or 'Reshape_1/shape' in node.name:
            b = node.attr['value'].tensor.tensor_content
            arr_int = [int.from_bytes(b[i:i + 4], 'little') for i in range(0, len(b), 4)]
            if len(arr_int):
                arr_byte = [unknown] + [struct.pack('<i', i) for i in arr_int[1:]]
                arr_byte = b''.join(arr_byte)
                node.attr['value'].tensor.tensor_content = arr_byte
            
            if len(node.attr['value'].tensor.int_val):
                node.attr['value'].tensor.int_val[0] = -1
    
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [26]:
g = load_graph('out/frozen_model.pb')

In [27]:
i = g.get_tensor_by_name('import/inputs:0')
o = g.get_tensor_by_name('import/SelectV2_3:0')
i, o

(<tf.Tensor 'import/inputs:0' shape=(?,) dtype=string>,
 <tf.Tensor 'import/SelectV2_3:0' shape=(?, 512) dtype=int32>)

In [28]:
test_sess = tf.Session(graph = g)

In [29]:
%%time

o_ = test_sess.run(o, feed_dict = {i: [f'ringkasan: {string}', 
                                       f'ringkasan: {string2}', 
                                       f'ringkasan: {string3}',
                                       f'ringkasan: {string4}',
                                       f'tajuk: {string}', 
                                       f'tajuk: {string2}', 
                                       f'tajuk: {string3}',
                                       f'tajuk: {string4}',
                                       'terjemah Inggeris ke Melayu: PETALING JAYA: Former prime minister Najib Razak has questioned whether the government knows how to manage the Covid-19 pandemic, outlining several seemingly contradictory announcements it has made.',
                                       'terjemah Melayu ke Inggeris: PETALING JAYA: Pertemuan bekas Perdana Menteri, Datuk Seri Najib Tun Razak dan Timbalan Perdana Menteri, Datuk Seri Ismail Sabri Yaakob hari ini adalah bagi membincangkan isu berkaitan hala tuju dan dasar negara.']})
o_

CPU times: user 1min 5s, sys: 11.1 s, total: 1min 16s
Wall time: 12 s


array([[11933,  1901,   747, ...,     0,     0,     0],
       [  725,  2442,   315, ...,     0,     0,     0],
       [ 5958, 10565,    37, ...,     0,     0,     0],
       ...,
       [   13,    12,   199, ...,     0,     0,     0],
       [18989,   541, 17554, ...,     0,     0,     0],
       [18989,   541, 17554, ...,     0,     0,     0]], dtype=int32)

In [30]:
import sentencepiece as spm
sp_model = spm.SentencePieceProcessor()
sp_model.Load(DEFAULT_SPM_PATH)

True

In [31]:
for k in range(len(o_)):
    print(k, sp_model.DecodeIds(o_[k].tolist()))

0 Amanah Kedah menyatakan tiada kenyataan daripada Adun Pakatan Harapan atau Bersatu negeri. Pengerusi Phahrolrazi berkata, kerajaan negeri masih berfungsi seperti biasa. Spekulasi mengenai pertukaran kerajaan telah menjadi kencang sejak semalam.
1 Kerajaan PH perlu pantau kenaikan harga barang basah atau mentah. Namun pada masa yang sama saya fikir elok juga jika kita imbas semula apa tindakan yang diambil oleh kerajaan PH.
2 Pemimpin Umno itu dilaporkan menggunakan ucapan titah yang di-Pertuan Agong. Beliau berkata, kerajaan yang ada sekarang adalah 'kerajaan Tuanku'. Kedua-dua pihak meminta PH menunggu sehingga PRU15 untuk mengambil alih kerajaan.
3 Jabatan Perkhidmatan Veteriner Putrajaya mengeluarkan kenyataan pada 9 Mei. Ia menjelaskan ayam ternakan tidak boleh dijangkiti virus Covid-19. Jurucakap jabatan itu berkata virus Covid-19 juga tidak mampu bertahan lama.
4 Amanah Kedah pertikai jika ada Adun Bersatu mahu lompat
5 'KPN perlu pantau kenaikan harga barang'
6 saran PH ambil 

In [ ]:
from tensorflow.tools.graph_transforms import TransformGraph

In [ ]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(minimum_size=1536000)',
             #'quantize_weights(fallback_min=-10240, fallback_max=10240)',
             'strip_unused_nodes',
             'sort_by_execution_order']

In [ ]:
pb = 'out/frozen_model.pb'
input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())
    
transformed_graph_def = TransformGraph(input_graph_def, 
       ['inputs'],
       ['SelectV2_3'], transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

In [ ]:
g = load_graph('out/frozen_model.pb.quantized')
i = g.get_tensor_by_name('import/inputs:0')
o = g.get_tensor_by_name('import/SelectV2_3:0')
i, o

In [ ]:
test_sess = tf.InteractiveSession(graph = g)

In [ ]:
o_ = test_sess.run(o, feed_dict = {i: [string3]})[0]

In [ ]:
sp_model.DecodeIds(o_.tolist())